In [1]:
import sys
sys.path.append("/home/tmartorella/ddpm")

from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from torchmetrics.image import StructuralSimilarityIndexMeasure
# from basicsr.metrics.metric_util import reorder_image, to_y_channel
from ddpm.datasets.celebahq import CelebAHQ
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2

t = transforms.Compose([transforms.ToTensor(), transforms.Resize([256]), transforms.CenterCrop([256, 256])])
# dataset = CelebAHQ(root="/home/bastien/scitas/bastien/CelebAMask-HQ/CelebA-HQ-img/", transform=t)

def to_pil(img):
    return Image.fromarray((img * 255).permute(1, 2, 0).numpy().astype(np.uint8))


# def _ssim(img1, img2):
#     """Calculate SSIM (structural similarity) for one channel images.

#     It is called by func:`calculate_ssim`.

#     Args:
#         img1 (ndarray): Images with range [0, 255] with order 'HWC'.
#         img2 (ndarray): Images with range [0, 255] with order 'HWC'.

#     Returns:
#         float: ssim result.
#     """

#     C1 = (0.01 * 255)**2
#     C2 = (0.03 * 255)**2

#     img1 = img1.astype(np.float64)
#     img2 = img2.astype(np.float64)
#     kernel = cv2.getGaussianKernel(11, 1.5)
#     window = np.outer(kernel, kernel.transpose())

#     mu1 = cv2.filter2D(img1, -1, window)[5:-5, 5:-5]
#     mu2 = cv2.filter2D(img2, -1, window)[5:-5, 5:-5]
#     mu1_sq = mu1**2
#     mu2_sq = mu2**2
#     mu1_mu2 = mu1 * mu2
#     sigma1_sq = cv2.filter2D(img1**2, -1, window)[5:-5, 5:-5] - mu1_sq
#     sigma2_sq = cv2.filter2D(img2**2, -1, window)[5:-5, 5:-5] - mu2_sq
#     sigma12 = cv2.filter2D(img1 * img2, -1, window)[5:-5, 5:-5] - mu1_mu2

#     ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
#     return ssim_map.mean()


# def calculate_ssim(img1, img2, crop_border, input_order='HWC', test_y_channel=False):
#     """Calculate SSIM (structural similarity).

#     Ref:
#     Image quality assessment: From error visibility to structural similarity

#     The results are the same as that of the official released MATLAB code in
#     https://ece.uwaterloo.ca/~z70wang/research/ssim/.

#     For three-channel images, SSIM is calculated for each channel and then
#     averaged.

#     Args:
#         img1 (ndarray): Images with range [0, 255].
#         img2 (ndarray): Images with range [0, 255].
#         crop_border (int): Cropped pixels in each edge of an image. These
#             pixels are not involved in the SSIM calculation.
#         input_order (str): Whether the input order is 'HWC' or 'CHW'.
#             Default: 'HWC'.
#         test_y_channel (bool): Test on Y channel of YCbCr. Default: False.

#     Returns:
#         float: ssim result.
#     """

#     assert img1.shape == img2.shape, (f'Image shapes are differnet: {img1.shape}, {img2.shape}.')
#     if input_order not in ['HWC', 'CHW']:
#         raise ValueError(f'Wrong input_order {input_order}. Supported input_orders are ' '"HWC" and "CHW"')
#     img1 = reorder_image(img1, input_order=input_order)
#     img2 = reorder_image(img2, input_order=input_order)
#     img1 = img1.astype(np.float64)
#     img2 = img2.astype(np.float64)

#     if crop_border != 0:
#         img1 = img1[crop_border:-crop_border, crop_border:-crop_border, ...]
#         img2 = img2[crop_border:-crop_border, crop_border:-crop_border, ...]

#     if test_y_channel:
#         img1 = to_y_channel(img1)
#         img2 = to_y_channel(img2)

#     ssims = []
#     for i in range(img1.shape[2]):
#         ssims.append(_ssim(img1[..., i], img2[..., i]))
#     return np.array(ssims).mean()

# def calculate_ssim(img1, img2, crop_border):
#     ssim = StructuralSimilarityIndexMeasure(data_range=255)
#     ssim(img1, img2)


# def get_metrics(img1, img2):
#     ssim = calculate_ssim(img1, img2, 0)
#     psnr = calculate_psnr(img1, img2, 0)
#     return ssim, psnr

In [2]:
from tqdm import tqdm
import torch
from torchmetrics.image import (
    PeakSignalNoiseRatio,
    StructuralSimilarityIndexMeasure,
)

from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity

def get_filtering_metrics(df):
    metrics = []
    
    psnr_tgt_metric = PeakSignalNoiseRatio().to("cuda")
    lpips_tgt_metric = LearnedPerceptualImagePatchSimilarity().to("cuda")

    for i, img in tqdm(df.iterrows(), total=len(df)):
        try:
            # src = t(Image.open(img["path"].parents[0 if img["algo"] == "none" else 2] / "corrupted.png")).to("cuda")
            tgt = t(Image.open(Path(img["path"]).parents[0 if img["algo"] == "none" else 2] / "original.png")).to("cuda")
            img = t(Image.open(img["path"])).to("cuda")

            
            # psnr_tgt_metric.update(img, tgt)
            # lpips_tgt_metric.update((img * 2 - 1).unsqueeze(0), (tgt * 2 - 1).unsqueeze(0))

            metrics.append({
                "psnr_to_target": psnr_tgt_metric(img, tgt).item(),
                "lpips_to_target": lpips_tgt_metric((img * 2 - 1).unsqueeze(0), (tgt * 2 - 1).unsqueeze(0)).item(),
            })  
        except Exception as e:
            print(f"error with image {i} - {e}")
    
    return pd.DataFrame(metrics)

def get_metrics(df, fid_metric, other_groups=[]):
    metrics = []
    
    for g in tqdm(df.groupby(["corruption"]+other_groups)):
        c = g[0]    
        
        psnr_src_metric = PeakSignalNoiseRatio().to("cuda")
        psnr_tgt_metric = PeakSignalNoiseRatio().to("cuda")
        ssim_src_metric = StructuralSimilarityIndexMeasure(data_range=(0, 1)).to("cuda")
        ssim_tgt_metric = StructuralSimilarityIndexMeasure(data_range=(0, 1)).to("cuda")
        lpips_src_metric = LearnedPerceptualImagePatchSimilarity().to("cuda")
        lpips_tgt_metric = LearnedPerceptualImagePatchSimilarity().to("cuda")
        l2_src_metric = []
        l2_tgt_metric = []
        fid_metric.reset()
        
        list_imgs = list(g[1].iterrows())
        for i, img in list_imgs:
            try:
                src = t(Image.open(Path(img["path"]).parents[0 if img["algo"] == "none" else 2] / "corrupted.png")).to("cuda")
                tgt = t(Image.open(Path(img["path"]).parents[0 if img["algo"] == "none" else 2] / "original.png")).to("cuda")
                img = t(Image.open(img["path"])).to("cuda")
                
                psnr_src_metric.update(img, src)
                psnr_tgt_metric.update(img, tgt)
                ssim_src_metric.update(img.unsqueeze(0), src.unsqueeze(0))
                ssim_tgt_metric.update(img.unsqueeze(0), tgt.unsqueeze(0))
                lpips_src_metric.update((img * 2 - 1).unsqueeze(0), (src * 2 - 1).unsqueeze(0))
                lpips_tgt_metric.update((img * 2 - 1).unsqueeze(0), (tgt * 2 - 1).unsqueeze(0))
                l2_src_metric.append(torch.linalg.norm((img - src).reshape(-1)))
                l2_tgt_metric.append(torch.linalg.norm((img - tgt).reshape(-1)))
                fid_metric.update((img * 255).to(torch.uint8).unsqueeze(0), real=False)
            except Exception as e:
                print(f"error with image {i} - {e}")
    
        psnr_to_source = psnr_src_metric.compute()
        psnr_to_target = psnr_tgt_metric.compute()
        ssim_to_source = ssim_src_metric.compute()
        ssim_to_target = ssim_tgt_metric.compute()
        l2_to_source = torch.stack(l2_src_metric).mean()
        l2_to_target = torch.stack(l2_tgt_metric).mean()
        lpips_to_source = lpips_src_metric.compute()
        lpips_to_target = lpips_tgt_metric.compute()
        fid = fid_metric.compute()
        
        metrics.append({
            "corruption": c[0],
            "psnr_to_input": psnr_to_source.item(),
            "psnr_to_target": psnr_to_target.item(),
            "ssim_to_input": ssim_to_source.item(),
            "ssim_to_target": ssim_to_target.item(),
            "l2_to_input": l2_to_source.item(),
            "l2_to_target": l2_to_target.item(),
            "lpips_to_input": lpips_to_source.item(),
            "lpips_to_target": lpips_to_target.item(),
            "fid": fid.item(),
            "num_imgs": len(list(g[1].iterrows()))
        })
        
    return pd.DataFrame(metrics)

In [8]:
from pathlib import Path
from tqdm.auto import tqdm
import os
import fnmatch

files = []
path_list = [
    "/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones",
    "/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba",
    "/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba2"
]

# for p in path_list:
#     files += [f for f in tqdm(list(Path(p).glob("**/*.png"))) if f.is_file()]
# files[:20]
files = []
for p in path_list:
    for root, dirnames, filenames in tqdm(os.walk(p)):
        for filename in fnmatch.filter(filenames, '*.png'):
            files.append(Path(os.path.join(root, filename)))
            if len(files) > 200:
                break
        if len(files) > 200:
            break
    if len(files) > 200:
        break

print(files[:20])

# files_scitas = []
# path_list = [
#     "/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones",
#     # "/home/bastien/rcp/scratch/vita-staff/users/bvandelft/results/ODEDIT/CELEBAHQ/Celeba"
# ]

# for p in path_list:
#     files_scitas += [f for f in tqdm(list(Path(p).glob("**/*.png"))) if f.is_file()]
# files[:20]

0it [00:00, ?it/s]

[PosixPath('/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones/2024-05-22_19-58/voronoi_noise/8991/original.png'), PosixPath('/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones/2024-05-22_19-58/voronoi_noise/8991/corrupted.png'), PosixPath('/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones/2024-05-22_19-58/voronoi_noise/8991/reconstruction/clipped/reconstruction_ddim.png'), PosixPath('/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones/2024-05-22_19-58/voronoi_noise/8991/ode/non_clipped/200_0.000316228_0.png'), PosixPath('/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones/2024-05-22_19-58/voronoi_noise/8991/ode/non_clipped/200_0.0001_0.png'), PosixPath('/home/bastien/scitas/bastien/results_CODE/results/ODEDIT/CELEBAHQ/fid/good_ones/2024-05-22_19-58/voronoi_noise/8991/ode/non_clipped/200_0.001_0.png'), PosixPath('/home/bastien/scitas/bastien/re

In [22]:
import pandas as pd
info = []
for f in files:
    f_split = str(f).split("/")
    if f.name in ["corrupted.png", "original.png"]:
        info.append({
            "path": f,
            "corruption": f_split[-3],
            "img_id": f_split[-2],
            "filename": f_split[-1],
            "algo": "none",
            "clipping": "none",
            "timestamp": f_split[-4],
            "latent": "none",
            "epsilon": np.nan,
        })
    else:
        info.append({
            "path": f,
            "corruption": f_split[-5],
            "img_id": f_split[-4],
            "filename": f_split[-1],
            "algo": f_split[-3],
            "clipping": f_split[-2],
            "timestamp": f_split[-6],
            "latent": f_split[-1].split("_")[0] if f_split[-3] == "sde" or f_split[-3] == "ode" else "none",
            "epsilon": f_split[-1].split("_")[1] if f_split[-3] == "ode" else np.nan
        })

df = pd.DataFrame(info)

In [23]:
df

,path,corruption,img_id,filename,algo,clipping,timestamp,latent,epsilon
0,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,original.png,none,none,2024-05-22_19-58,none,NaN
1,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,corrupted.png,none,none,2024-05-22_19-58,none,NaN
2,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,reconstruction_ddim.png,reconstruction,clipped,2024-05-22_19-58,none,NaN
3,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,200_0.000316228_0.png,ode,non_clipped,2024-05-22_19-58,200,0.000316228
4,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,200_0.0001_0.png,ode,non_clipped,2024-05-22_19-58,200,0.0001
...,...,...,...,...,...,...,...,...,...
196,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,10702,200_0.0001_0.png,ode,non_clipped,2024-05-22_19-58,200,0.0001
197,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,10702,200_0.001_0.png,ode,non_clipped,2024-05-22_19-58,200,0.001
198,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,13228,original.png,none,none,2024-05-22_19-58,none,NaN
199,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,13228,corrupted.png,none,none,2024-05-22_19-58,none,NaN


In [30]:
df.loc[df['path'].apply(lambda x: 'scitas' in str(x)) & df['timestamp'].isin(['2024-05-22_19-52', '2024-05-22_19-58']) & (df['algo'] == 'ode'), 'clipping'] = 'clipped'
df


,path,corruption,img_id,filename,algo,clipping,timestamp,latent,epsilon
0,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,original.png,none,none,2024-05-22_19-58,none,NaN
1,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,corrupted.png,none,none,2024-05-22_19-58,none,NaN
2,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,reconstruction_ddim.png,reconstruction,clipped,2024-05-22_19-58,none,NaN
3,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,200_0.000316228_0.png,ode,clipped,2024-05-22_19-58,200,0.000316228
4,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,8991,200_0.0001_0.png,ode,clipped,2024-05-22_19-58,200,0.0001
...,...,...,...,...,...,...,...,...,...
196,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,10702,200_0.0001_0.png,ode,clipped,2024-05-22_19-58,200,0.0001
197,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,10702,200_0.001_0.png,ode,clipped,2024-05-22_19-58,200,0.001
198,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,13228,original.png,none,none,2024-05-22_19-58,none,NaN
199,/home/bastien/scitas/bastien/results_CODE/resu...,voronoi_noise,13228,corrupted.png,none,none,2024-05-22_19-58,none,NaN


In [6]:
df[(df['algo'] == "ode") & (df['clipping'] == 'non_clipped')].groupby("corruption")['img_id'].unique().apply(len)

corruption
elastic_transform    4
frost                4
gaussian_blur        4
gaussian_noise       4
jpeg_compression     2
pixelate             4
shot_noise           4
Name: img_id, dtype: int64

In [7]:
df[(df['algo'] == "ode") & (df['clipping'] == 'clipped')].groupby("corruption")['img_id'].unique().apply(len)

corruption
elastic_transform    4
frost                4
gaussian_blur        4
gaussian_noise       4
jpeg_compression     2
pixelate             4
shot_noise           4
Name: img_id, dtype: int64

In [8]:
df[(df['algo'] == "sde") & (df['latent'] == "699")].groupby("corruption")['img_id'].unique().apply(len)

Series([], Name: img_id, dtype: object)

In [9]:
df[(df['algo'] == "sde") & (df['latent'] == "299")].groupby("corruption")['img_id'].unique().apply(len)

Series([], Name: img_id, dtype: object)

In [10]:
df['corruption'].unique()

array(['pixelate', 'gaussian_blur', 'elastic_transform', 'shot_noise',
       'gaussian_noise', 'frost', 'jpeg_compression'], dtype=object)

In [11]:
df_with_metrics = pd.concat([df, get_filtering_metrics(df)], axis=1)

  0%|          | 0/201 [00:00<?, ?it/s]

In [12]:
df_with_metrics.to_csv("./df_with_metrics.csv")
df_with_metrics

,path,corruption,img_id,filename,algo,clipping,timestamp,latent,epsilon,psnr_to_target,lpips_to_target
0,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,corrupted.png,none,none,2024-07-04_11-45,none,NaN,26.589617,0.367076
1,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,original.png,none,none,2024-07-04_11-45,none,NaN,inf,0.000000
2,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,200_0.002_0.png,ode,clipped,2024-07-04_11-45,200,0.0010,19.367126,0.355870
3,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,200_0.0001_0.png,ode,non_clipped,2024-07-04_11-45,200,0.0001,25.478458,0.121347
4,/home/bastien/rcp/scratch/vita-staff/users/bva...,pixelate,15053,200_0.000271442_0.png,ode,non_clipped,2024-07-04_11-45,200,0.0001,24.356153,0.142400
...,...,...,...,...,...,...,...,...,...,...,...
196,/home/bastien/rcp/scratch/vita-staff/users/bva...,jpeg_compression,29137,original.png,none,none,2024-07-04_11-45,none,NaN,inf,0.000000
197,/home/bastien/rcp/scratch/vita-staff/users/bva...,jpeg_compression,29137,200_0.002_0.png,ode,clipped,2024-07-04_11-45,200,0.0010,18.954708,0.185596
198,/home/bastien/rcp/scratch/vita-staff/users/bva...,jpeg_compression,29137,200_0.002_0.png,ode,non_clipped,2024-07-04_11-45,200,0.0010,19.619682,0.191704
199,/home/bastien/rcp/scratch/vita-staff/users/bva...,jpeg_compression,29137,reconstruction_ddim.png,reconstruction,clipped,2024-07-04_11-45,none,NaN,24.781080,0.196950


# top 1 and top 4 by psnr

In [13]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(1), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_psnr.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(4), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_psnr.csv")
display(df_with_metrics_filtered.head(5))

,img_id,corruption,algo,clipping,path,filename,timestamp,latent,epsilon,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.005781,0.284392
2,12179,frost,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,11.059509,0.451597
3,12179,frost,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,19.441391,0.341820


,img_id,corruption,algo,clipping,path,filename,timestamp,latent,epsilon,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.005781,0.284392
2,12179,frost,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,11.059509,0.451597
3,12179,frost,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,19.441391,0.341820


# top 1 and top 4 by lpips

In [14]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(1), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_lpips.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping'])
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(4), include_groups=False)
    .reset_index().drop(columns="level_4")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_lpips.csv")
display(df_with_metrics_filtered.head(5))

,img_id,corruption,algo,clipping,path,filename,timestamp,latent,epsilon,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.005781,0.284392
2,12179,frost,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,11.059509,0.451597
3,12179,frost,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,19.441391,0.341820


,img_id,corruption,algo,clipping,path,filename,timestamp,latent,epsilon,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,20.005781,0.284392
2,12179,frost,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,11.059509,0.451597
3,12179,frost,ode,non_clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,200,0.001,19.441391,0.341820


# top 1 and top 4 by psnr per latent and epsilon

In [15]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(1), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_psnr_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('psnr_to_target', ascending=False).head(4), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_psnr_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))

,img_id,corruption,algo,clipping,latent,epsilon,path,filename,timestamp,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.005781,0.284392
2,12179,frost,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,11.059509,0.451597
3,12179,frost,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,19.441391,0.341820


,img_id,corruption,algo,clipping,latent,epsilon,path,filename,timestamp,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.005781,0.284392
2,12179,frost,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,11.059509,0.451597
3,12179,frost,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,19.441391,0.341820


# top 1 and top 4 by lpips by latent and epsilon

In [16]:
######################
# now top 1

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(1), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_1_imgs_by_lpips_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))


#########################
# now top 4

df_with_metrics_filtered = (
    df_with_metrics[~df_with_metrics['algo'].isin(["none", "reconstruction"])]
    .groupby(['img_id', 'corruption', 'algo', 'clipping', 'latent', 'epsilon'], observed=False)
    .apply(lambda x: x.sort_values('lpips_to_target', ascending=True).head(4), include_groups=False)
    .reset_index().drop(columns="level_6")
)

df_with_metrics_filtered = pd.concat([
    df_with_metrics_filtered,
    df_with_metrics[df_with_metrics['algo'].isin(['none', 'reconstruction'])]
], axis=0)

df_with_metrics_filtered.to_csv("./top_4_imgs_by_lpips_by_latent_and_epsilon.csv")
display(df_with_metrics_filtered.head(5))

,img_id,corruption,algo,clipping,latent,epsilon,path,filename,timestamp,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.005781,0.284392
2,12179,frost,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,11.059509,0.451597
3,12179,frost,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,19.441391,0.341820


,img_id,corruption,algo,clipping,latent,epsilon,path,filename,timestamp,psnr_to_target,lpips_to_target
0,12179,elastic_transform,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.315748,0.307156
1,12179,elastic_transform,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,20.005781,0.284392
2,12179,frost,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,11.059509,0.451597
3,12179,frost,ode,non_clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,10.483303,0.566133
4,12179,gaussian_blur,ode,clipped,200,0.001,/home/bastien/rcp/scratch/vita-staff/users/bva...,200_0.002_0.png,2024-07-04_11-45,19.441391,0.341820


# Calculate Metrics

In [17]:
import pandas as pd

df = pd.read_csv('./top_4_imgs_by_psnr_by_latent_and_epsilon.csv')

In [18]:
dfs = {
    "reconstruction_without_clipping": {
        "df": df[(df["algo"] == "reconstruction") & (df["clipping"] == "non_clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "reconstruction_with_clipping": {
        "df": df[(df["algo"] == "reconstruction") & (df["clipping"] == "clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "ode_without_clipping": {
        "df": df[(df["algo"] == "ode") & (df["clipping"] == "non_clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "ode_with_clipping": {
        "df": df[(df["algo"] == "ode") & (df["clipping"] == "clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "sde_without_clipping": {
        "df": df[(df["algo"] == "sde") & (df["clipping"] == "non_clipped")].copy().drop(columns=["timestamp"]).copy()
    },
    "original": {
        "df": df[(df["algo"] == "none") & (df["filename"] == "original.png")].copy().drop(columns=["timestamp"]).copy()
    },
    "corrupted": {
        "df": df[(df["algo"] == "none") & (df["filename"] == "corrupted.png")].copy().drop(columns=["timestamp"]).copy()
    },
}

In [19]:

import sys
sys.path.append("/home/tmartorella/ddpm")

from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
from torchmetrics.image import StructuralSimilarityIndexMeasure
# from basicsr.metrics.metric_util import reorder_image, to_y_channel
from ddpm.datasets.celebahq import CelebAHQ
from torchvision import transforms
from PIL import Image
import numpy as np
import cv2

celebahq_fid = FrechetInceptionDistance(reset_real_features=False).to("cuda")

import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from ddpm.datasets.celebahq import CelebAHQ
from torchvision import transforms
t =  transforms.Compose([transforms.ToTensor(), transforms.Resize([256]), transforms.CenterCrop([256, 256])])

class ADataset(Dataset):
    def __init__(self, ds):
        self.ds = ds
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        return self.ds[idx][0]

celeba_dataset = CelebAHQ(root="/home/bastien/rcp/scratch/datasets/CelebAMask-HQ/CelebA-HQ-img/", transform=t)
loader = DataLoader(ADataset(celeba_dataset), batch_size=16, num_workers=4, pin_memory=True)

for batch in tqdm(loader):
    celebahq_fid.update((batch * 255).to(torch.uint8).to("cuda"), real=True)

100%|███████████████████████████████████████████████████████████████████████████████| 1875/1875 [07:39<00:00,  4.08it/s]


In [20]:
from IPython.display import display, Markdown

for k, v in dfs.items():
    print(k)
    
    metrics_per_corruption = get_metrics(v["df"].copy(), celebahq_fid)
    display(Markdown(f"## {k}"))
    display(metrics_per_corruption)
    metrics_per_corruption.to_csv(f"./metrics_per_corruption/{k}_per_corruption.csv")

    v["metrics_per_corruption"] = metrics_per_corruption

    _fake_df = v["df"].copy()

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid, ['epsilon', 'latent'])
    display(Markdown(f"## {k} - aggregated"))
    display(metrics_aggregated)
    metrics_per_corruption.to_csv(f"./metrics_per_epsilon-latent/{k}_aggregated.csv")
    
    v["metrics_per_corruption_and_epsilon_and_latent"] = metrics_aggregated

    _fake_df = v["df"].copy()
    _fake_df["corruption"] = "all"

    metrics_aggregated = get_metrics(_fake_df.copy(), celebahq_fid)
    display(Markdown(f"## {k} - aggregated"))
    display(metrics_aggregated)
    metrics_per_corruption.to_csv(f"./metrics_aggregated/{k}_aggregated.csv")
    
    v["metrics_aggregated"] = metrics_aggregated

reconstruction_without_clipping


100%|█████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:19<00:00,  2.78s/it]


## reconstruction_without_clipping

,corruption,psnr_to_input,psnr_to_target,ssim_to_input,ssim_to_target,l2_to_input,l2_to_target,lpips_to_input,lpips_to_target,fid,num_imgs
0,elastic_transform,27.138981,18.921211,0.857441,0.474952,16.951801,49.614239,0.166865,0.336186,313.690948,5
1,frost,25.706547,8.180790,0.879893,0.338636,18.296045,165.945587,0.168553,0.643489,355.889465,5
2,gaussian_blur,34.989956,22.135277,0.923606,0.590980,7.266910,34.294384,0.153953,0.482706,238.412231,5
3,gaussian_noise,42.029514,10.586029,0.999579,0.056449,3.510104,131.055176,0.000534,1.319231,388.257233,5
4,jpeg_compression,35.590237,24.753370,0.947064,0.660769,6.812136,25.246908,0.039750,0.265602,240.833176,2
5,pixelate,32.324783,24.643635,0.925542,0.657241,8.694854,25.733875,0.079879,0.406044,266.501678,5
6,shot_noise,44.525661,10.541932,0.999639,0.081740,2.631741,130.533768,0.000346,1.231099,354.804443,5


OSError: Cannot save file into a non-existent directory: 'metrics_per_corruption'

# Fin fin